In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [ ]:
model = SentenceTransformer('all-MiniLM-L12-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
sds = pd.read_csv("data/small_dataset")

In [ ]:
embeddings = model.encode(sds['lyrics'])

In [ ]:
embeddings

array([[-1.31038964e-01,  1.82835609e-02, -3.78568582e-02, ...,
         6.01930224e-05,  2.52264198e-02, -1.38734495e-02],
       [-5.06527498e-02,  3.69102024e-02,  1.98155567e-02, ...,
         4.73509207e-02,  1.69067178e-02,  1.62009802e-02],
       [-5.76839410e-02,  4.75641787e-02,  1.90364628e-03, ...,
         8.02092180e-02,  5.36910538e-03, -5.62414564e-02],
       ...,
       [-5.58314333e-03,  1.10310338e-01, -3.39260958e-02, ...,
         7.67268389e-02,  3.38497083e-03, -3.25751267e-02],
       [-4.95762657e-03, -5.17929643e-02,  1.16702043e-01, ...,
         4.69979048e-02, -5.37207872e-02,  1.70569289e-02],
       [ 3.40427319e-03,  6.75700605e-02,  6.94579706e-02, ...,
        -4.65298779e-02,  3.38704437e-02,  1.26633956e-03]], dtype=float32)

In [ ]:
sds['embeddings'] = list(embeddings)

In [ ]:
def cosine_similarity(v1, v2):
    d = np.dot(v1, v2)
    cos_theta = d / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return(cos_theta)

In [ ]:
def relevance_scores(query_embed):
    scores = [cosine_similarity(query_embed, v2) for v2 in sds['embeddings']]
    scores = pd.Series(scores)
    return(scores)

In [ ]:
def semantic_search(query_sentence, df = sds, return_top = False):
    query_embed = model.encode(query_sentence)
    scores = relevance_scores(query_embed)
    df['scores'] = scores
    sorted_df = df.sort_values(by = 'scores', ascending = False)
    if return_top == False:
        return sorted_df
    else:
        return sorted_df.iloc[0]['lyrics']

In [ ]:
semantic_search("i'm happy you are doing well after our breakup")

,Unnamed: 0,index,title,tag,artist,year,views,features,lyrics,id,embeddings,scores
1313,164519,3959866,​happier,pop,Olivia Rodrigo,2021,1230015,{},\nWe broke up a month ago\nYour friends are mi...,5416223,"[-0.050075706, -0.069140814, 0.10742216, -0.00...",0.598926
694,79571,1841524,Let Me Love You,pop,Ariana Grande,2016,1090086,"{""Lil Wayne""}",\nI just broke up with my ex\nNow I'm out here...,2437285,"[-0.018333275, -0.0673361, 0.1049746, -0.03285...",0.576452
1375,178971,4981757,​good 4 u,pop,Olivia Rodrigo,2021,2486863,{},"\n(Ah)\n\n\nWell, good for you, I guess you mo...",6702232,"[-0.07039017, -0.010325448, 0.11081872, 0.0729...",0.555228
189,12198,61668,Somebody That I Used to Know,pop,Gotye,2011,1737049,{Kimbra},\nNow and then I think of when we were togethe...,65848,"[-0.0443811, -0.017374398, 0.088465914, 0.0406...",0.432752
711,81127,1859609,Closer,pop,The Chainsmokers,2016,6721409,{Halsey},"\nHey, I was doing just fine before I met you\...",2458848,"[-0.01597903, 0.0048104706, 0.11487218, 0.0246...",0.430104
...,...,...,...,...,...,...,...,...,...,...,...,...
1151,129211,2678866,Nico And The Niners,pop,twenty one pilots,2018,1087923,"{""​twenty one pilots""}",\nMsilaiv ecnuoned ew\nTsae eurt daeh dna amed...,3812599,"[-0.03603754, 0.026914828, -0.058671877, -0.04...",-0.039652
693,79553,1841298,How Genius Works,misc,Genius,2016,1331958,{},Genius is the ultimate source of music knowled...,2437045,"[0.017587233, -0.10442876, 0.037475057, 0.0212...",-0.042989
1232,144280,3140514,Au DD,rap,PNL,2019,1270655,{},\n\n\nBats les couilles d'l'Himalaya\nBats les...,4388058,"[-0.013177911, 0.0035447918, -0.07363458, -0.0...",-0.047094
67,875,1046,Sing for the Moment,rap,Eminem,2002,1177104,{},\nThese ideas are nightmares to white parents\...,1010,"[0.02095876, 0.056278836, -0.017212084, -0.043...",-0.049496


In [ ]:
print(semantic_search("i'm happy you are doing well after our breakup", return_top = True))


We broke up a month ago
Your friends are mine, you know I know
You've moved on, found someone new
One more girl who brings out the better in you
And I thought my heart was detached
From all the sunlight of our past
But she's so sweet, she's so pretty
Does she mean you forgot about me?


Oh, I hope you're happy
But not like how you were with me
I'm selfish, I know, I can't let you go
So find someone great, but don't find no one better
I hope you're happy, but don't be happier


And do you tell her she's the most beautiful girl you've ever seen?
An eternal love bullshit you know you'll never mean
Remember when I believed
You meant it when you said it first to me?
And now I'm pickin' her apart
Like cuttin' her down will make you miss my wretched heart
But she's beautiful, she looks kind
She probably gives you butterflies

I hope you're happy
But not like how you were with me
I'm selfish, I know, I can't let you go
So find someone great, but don't find no one better
I hope you're happy
I 